In [1]:
pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 KB 3.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import category_encoders

In [ ]:
#hotels = pd.read_csv('data/hotels.csv')
#hotels.head(3)

In [ ]:
#hotels.duplicated().value_counts()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/proj3/демо/data/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv('/content/drive/MyDrive/proj3/демо/data/hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv('/content/drive/MyDrive/proj3/демо/data/submission.csv')

In [ ]:
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 

In [ ]:
hotels = df_test.append(df_train, sort=False).reset_index(drop=True) # Объеденяем наши базы

In [ ]:
hotels.head(5) # Беглый взгляд

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng,sample,reviewer_score
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,1,"[' Leisure trip ', ' Couple ', ' Double Room '...",13 days,45.533137,9.171102,0,0.0
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,12/12/2016,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,0,7,"[' Leisure trip ', ' Group ', ' Triple Room ',...",234 day,52.385649,4.834443,0,0.0
2,Mallorca 251 Eixample 08008 Barcelona Spain,46,11/26/2015,8.3,Alexandra Barcelona A DoubleTree by Hilton,Sweden,Pillows,3,351,Nice welcoming and service,5,15,"[' Business trip ', ' Solo traveler ', ' Twin ...",616 day,41.393192,2.161520,0,0.0
3,Piazza Della Repubblica 17 Central Station 201...,241,10/17/2015,9.1,Hotel Principe Di Savoia,United States of America,No Negative,0,1543,Everything including the nice upgrade The Hot...,27,9,"[' Leisure trip ', ' Couple ', ' Ambassador Ju...",656 day,45.479888,9.196298,0,0.0
4,Singel 303 309 Amsterdam City Center 1012 WJ A...,834,5/16/2016,9.1,Hotel Esther a,United Kingdom,No Negative,0,4687,Lovely hotel v welcoming staff,7,2,"[' Business trip ', ' Solo traveler ', ' Class...",444 day,52.370545,4.888644,0,0.0


In [ ]:
hotels.info() # Изучяем какие признаки у нас есть в самом начале, видим что у двух признаков lat/lng есть пропуски которые будем заполнять.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 18 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               515738 non-null  object 
 1   additional_number_of_scoring                515738 non-null  int64  
 2   review_date                                 515738 non-null  object 
 3   average_score                               515738 non-null  float64
 4   hotel_name                                  515738 non-null  object 
 5   reviewer_nationality                        515738 non-null  object 
 6   negative_review                             515738 non-null  object 
 7   review_total_negative_word_counts           515738 non-null  int64  
 8   total_number_of_reviews                     515738 non-null  int64  
 9   positive_review                             515738 non-null  object 
 

In [ ]:
hotels.duplicated().value_counts() # Смотрим сколько у нас дубликатов

False    515402
True        336
dtype: int64

In [ ]:
hotels = hotels.drop_duplicates() # Очищаем от полных дубликатов


Заполнять отсутсвующие значения решил с помощью геокодера от Яндекс

In [ ]:
# что бы не нужно было каждый раз прогружать через геокодер, сохранил полученые значения в отдельный файл.

#from yandex_geocoder import Client

#key = pd.read_table('key.txt') # личный ключ к доступу на сервис спрятал в отдельный файл
#client = Client(key)

#df = pd.DataFrame() #Создаем отедльный фрейм в котором будем искать пропущенные координаты
#df['hotel_address'] = hotels['hotel_address'] # для поиска координат взял из основного фрейма адрес и координаты
#df['lat'] = hotels['lat'].isnull()
#df['lng'] = hotels['lng'].isnull()

#df = df[(df['lat'] == True) & (df['lng'] == True)]

#def coordinates_lat (lat_):
#    coordinates = client.coordinates(lat_)
#    return coordinates

#df['lat_lng'] = df['hotel_address'].apply(coordinates_lat)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/proj3/демо/coordinat.csv') # Подготовленный фрейм с координатами.

In [ ]:
df.head(5)

,Unnamed: 0,hotel_address,lat,lng,lat_lng
0,122,Savoyenstra e 2 16 Ottakring 1160 Vienna Austria,True,True,"(Decimal('16.297308'), Decimal('48.211783'))"
1,566,23 Rue Damr mont 18th arr 75018 Paris France,True,True,"(Decimal('2.351556'), Decimal('48.856663'))"
2,724,Josefst dter Stra e 10 12 08 Josefstadt 1080 V...,True,True,"(Decimal('16.347119'), Decimal('48.210918'))"
3,754,W hringer Stra e 33 35 09 Alsergrund 1090 Vien...,True,True,"(Decimal('16.357513'), Decimal('48.222139'))"
4,1137,4 rue de la P pini re 8th arr 75008 Paris France,True,True,"(Decimal('2.351556'), Decimal('48.856663'))"


Удаляем уже не нужные признаки в новом фрейм с координатами.

In [ ]:
df = df.drop(['lat'], axis = 1)
df = df.drop(['lng'], axis = 1)
df = df.drop(['hotel_address'], axis = 1)

Подчистим результаты признака с новыми координатами

In [ ]:
def lat_lng(agr):
    i = str(agr) # Преобразуем в строку
    res = i.replace(' ', '') # Удалим пробелы что бы в дальнейшем можно было преобразовать в список
    return res[1:-1] # Возвращаем результат без внешних скобок

In [ ]:
df['lat_lng'] = df['lat_lng'].apply(lat_lng) # Применяем нашу функцию

In [ ]:
df

,Unnamed: 0,lat_lng
0,122,"Decimal('16.297308'),Decimal('48.211783')"
1,566,"Decimal('2.351556'),Decimal('48.856663')"
2,724,"Decimal('16.347119'),Decimal('48.210918')"
3,754,"Decimal('16.357513'),Decimal('48.222139')"
4,1137,"Decimal('2.351556'),Decimal('48.856663')"
...,...,...
2443,386092,"Decimal('16.39411'),Decimal('48.218213')"
2444,386465,"Decimal('16.39411'),Decimal('48.218213')"
2445,386504,"Decimal('2.351556'),Decimal('48.856663')"
2446,386702,"Decimal('16.39411'),Decimal('48.218213')"


Ниже созданы две функция одна из которых возвращает значение lat, а другая lng

In [ ]:
# Ниже созданы две функции которые преобразуют строку в список по запятой, и одна из функий возвращает значение lat, другая lng
def lat_ (agr):
    list_cor = agr.split(',')
    return list_cor[1]
def lng_ (agr):
    list_cor = agr.split(',')
    return list_cor[0]

In [ ]:
# Применяем наши функции, и создаем новые признаки
df['lat'] = df['lat_lng'].apply(lat_)
df['lng'] = df['lat_lng'].apply(lng_)
df = df.drop('lat_lng', axis=1)

In [ ]:
import re

In [ ]:
# Создадим функцию использующую регулярное вырожение для поиск чисел в наших признака
def corr (agr):
    res = re.findall(r'\d*\.\d*', agr) # Ищем числа до точки и после
    result = str(res)
    return result[2:-2] # Результат получается в квадртных скобках и ковычках, убираем их, что бы осталось чистое число

In [ ]:
# Применяем нашу функцию
df['lat'] = df['lat'].apply(corr)
df['lng'] = df['lng'].apply(corr)

In [ ]:
df

,Unnamed: 0,lat,lng
0,122,48.211783,16.297308
1,566,48.856663,2.351556
2,724,48.210918,16.347119
3,754,48.222139,16.357513
4,1137,48.856663,2.351556
...,...,...,...
2443,386092,48.218213,16.39411
2444,386465,48.218213,16.39411
2445,386504,48.856663,2.351556
2446,386702,48.218213,16.39411


In [ ]:
df = df.set_index('Unnamed: 0') # Делаем признак 'Unnamed: 0' индексом фрейма.

In [ ]:
hotels = hotels.fillna(df) # Заполняем пропущенные значения в основном фрейме данных из отдельно созданного фрейма в котором добавили новые координаты.

Возьмемся за следующий признак указывающий город в котором расположен отель.

In [ ]:
# посмотрим внимательнее на адреса, и отметим для себя что предпоследним словом является город, за исключением Великобритании, в них город указан на 5 месте с конца.
display(hotels['hotel_address'][0])
display(hotels['hotel_address'][1])
display(hotels['hotel_address'][2])
display(hotels['hotel_address'][3])
display(hotels['hotel_address'][4])
display(hotels['hotel_address'][5])

'Via Senigallia 6 20161 Milan Italy'

'Arlandaweg 10 Westpoort 1043 EW Amsterdam Netherlands'

'Mallorca 251 Eixample 08008 Barcelona Spain'

'Piazza Della Repubblica 17 Central Station 20124 Milan Italy'

'Singel 303 309 Amsterdam City Center 1012 WJ Amsterdam Netherlands'

'Coram Street Camden London WC1N 1HT United Kingdom'

In [ ]:
def city(agr):
    i = agr.split(' ') # Разделяем слова по пробелу
    result = i[-2] # Возвращаем предпоследние значение
    if result == 'United': # учитываем что если попадается Великобритания, то возвращаем 5 число с конца
        return i[-5]
    else:
        return i[-2]

In [ ]:
hotels['City'] = hotels['hotel_address'].apply(city)

In [ ]:
hotels['City'].value_counts()

London       262299
Barcelona     60149
Paris         59598
Amsterdam     57212
Vienna        38938
Milan         37206
Name: City, dtype: int64

In [ ]:
#hotels = hotels.drop(['hotel_address'], axis=1) # Удаляем теперь не нужный нам признак с адресом

Мной было решено создать признак указывающий на расстояние отеля от центра города в котором он расположен, для этого у нас есть координаты отеля, и город в котором он расположен, осталось взять из интернета координаты центра города, так как городов у нас всего лишь 6, в ручную будет сделать быстрее.

In [ ]:
from math import sin, cos, sqrt, atan2, radians # Импортируем библиотеки которые будут счиать по формуле расстояние отеля от центра города
# Создаем словарь в котором будет указан город и координаты его центра
City_centre = {
    'London':(51.507351, -0.127696),
    'Barcelona':(41.392696, 2.144449),
    'Paris':(48.856663, 2.351556),
    'Amsterdam':(52.373057, 4.892557),
    'Vienna':(48.206481, 16.363451),
    'Milan':(45.464183, 9.189595)
}
# создаем функцию для вычислений
def distance_to_centre(arg):
    city = arg['City']
    lat = arg['lat']
    lng = arg['lng']
    R = 6373.0

    # Координаты первой точки
    lat1 = radians(lat)
    lon1 = radians(lng)
    # Координаты второй точки
    lat2 = radians(City_centre[city][0])
    lon2 = radians(City_centre[city][1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return round(distance, 1)

In [ ]:
# Перед тем как применять нашу функцию, что бы избежать ошибок, приравняем все значения в признаках координат к типу данных float
def to_float(agr):
    res = float(agr)
    return res

In [ ]:
hotels['lat'] = hotels['lat'].apply(to_float)
hotels['lng'] = hotels['lng'].apply(to_float)

In [ ]:
hotels['distance_to_centre_in_km'] = hotels.apply(distance_to_centre, axis=1)

In [ ]:
import plotly.express as px

In [ ]:
fig = px.histogram(
    data_frame=hotels,
    x='distance_to_centre_in_km',
    title='Распределение расстояние до центра города',
    histnorm='percent',
    width=1000,
    marginal='box',
)
#fig.write_html("plotly/histogram1.html")

fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
i = [ 22933,  38905,  62759, 368808]
hotels.iloc[i]

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng,sample,reviewer_score,City,distance_to_centre_in_km
22933,4 rue de la P pini re 8th arr 75008 Paris France,172,4/10/2017,8.8,Maison Albar Hotel Paris Op ra Diamond,United Kingdom,Not much,3,1524,Nice view if you get an upper floor pointing ...,13,3,"[' Business trip ', ' Family with young childr...",115 day,48.222139,16.357513,0,0.0,Paris,1032.4
38906,Josefst dter Stra e 22 08 Josefstadt 1080 Vien...,39,3/14/2016,8.4,Cordial Theaterhotel Wien,France,No Negative,0,422,The breakfast is very good Sweet and savory M...,20,2,"[' Leisure trip ', ' Couple ', ' Double Room '...",507 day,48.856663,2.351556,0,0.0,Vienna,1033.1
62765,W hringer Stra e 33 35 09 Alsergrund 1090 Vien...,214,5/6/2016,7.8,Hotel Atlanta,Canada,We found the bed a little hard A little close...,29,2724,The large room It was quiet The staff was hel...,17,1,"[' Leisure trip ', ' Couple ', ' Double or Twi...",454 day,48.856663,2.351556,0,0.0,Vienna,1033.1
368945,Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Au...,211,4/21/2016,8.6,Hotel City Central,Norway,Nothing,3,2923,Everything was great we booked a standard roo...,33,43,"[' Leisure trip ', ' Couple ', ' Standard Twin...",469 day,48.856663,2.351556,1,10.0,Vienna,1033.1


In [ ]:
mean_dist = hotels.groupby('City', as_index=False)['distance_to_centre_in_km'].mean()

In [ ]:
mean_dist

,City,distance_to_centre_in_km
0,Amsterdam,2.455093
1,Barcelona,2.700112
2,London,3.886552
3,Milan,2.662366
4,Paris,2.888531
5,Vienna,1.959456


In [ ]:
city = hotels.iloc[ind]

In [ ]:
for ind in i:
      inde = hotels['City'].iloc[ind]
      print(inde)

Paris
Vienna
Vienna
Vienna


In [ ]:
city

In [ ]:
for ind in i:
    city = hotels['City'].iloc[ind]
      
    mean_dist = hotels.groupby(city, as_index=False)['distance_to_centre_in_km'].mean()
    hotels['distance_to_centre_in_km'].iloc[ind] = mean_dist

KeyError: ignored

In [ ]:
print(hotels[hotels['distance_to_centre_in_km']>18])

        additional_number_of_scoring review_date average_score  \
22933                            172   4/10/2017           8.8   
38906                             39   3/14/2016           8.4   
62765                            214    5/6/2016           7.8   
368945                           211   4/21/2016           8.6   

                                    hotel_name reviewer_nationality  \
22933   Maison Albar Hotel Paris Op ra Diamond      United Kingdom    
38906                Cordial Theaterhotel Wien              France    
62765                            Hotel Atlanta              Canada    
368945                      Hotel City Central              Norway    

                                          negative_review  \
22933                                            Not much   
38906                                         No Negative   
62765    We found the bed a little hard A little close...   
368945                                           Nothing    

       

In [ ]:
# Так как города у нас это категориальный признак, закодируем его. 
hotels = pd.get_dummies(hotels, columns=['City'])

In [ ]:
# Теперь удаляем не нужные признаки с координатами 
hotels = hotels.drop(['lat'], axis=1)
hotels = hotels.drop(['lng'], axis=1)

Далее займемся названиями отелей

In [ ]:
# Создадим список/счетчик всех слов, после чего, создадим список самых часто встречающихся слов.
list_tag_in_hotel_name = [] # Список популярных слов

full_tag_in_hotel_name = {} # Словарь всех слов

# Напишем функцию приводящую к нижнему регистру(на всякий случай), и создающая список по пробелу, и добавляющая каждое уникальное слово и кол-во его повторений во всех названий отелей
def hotel_name_remote(agr):
    name = agr.lower()
    res = name.split(' ')
    for i in res:
        if i not in full_tag_in_hotel_name:
            full_tag_in_hotel_name[i] = 1
        else:
            full_tag_in_hotel_name[i] +=1

hotels['hotel_name'].apply(hotel_name_remote)

# Так как слов очень много, оставим только те, которые встречаются больше 10000 раз
for i in full_tag_in_hotel_name:
    if full_tag_in_hotel_name[i] > 10000:
        list_tag_in_hotel_name.append(i)

print(len(list_tag_in_hotel_name))

34


In [ ]:
# Оставим только слова которые не привязанны к городам, и имеют смысл.
list_tag_in_hotel_name = [
 'hilton',
 'holiday',
 'park',
 'plaza',
 'best',
 'western',
 'palace',
 'city',
 'tower',
 'grand',
 'radisson',
 'blu'
]

In [ ]:
# Напишем функцию которая будет брать поочередно слово из названия отеля и проверять, находится ли оно в списке топовых слов, и возвращать только слова которые наиболее популярны.
def tag_in_hotel_name(agr):
    name = agr.lower()
    res = name.split(' ')
    result = []
    for i in res:
        if i in list_tag_in_hotel_name:
            if i not in result:
                result.append(i)
    if len(result) == 0: # добавил проверку на то есть ли в названии отеля популрные слова, если нет, возвращать пустое значение.
        return None
    else:
        return result
    
# применяем нашу функцию и сразу удаляем обработанный признак.
hotels['tag_in_hotel_name'] = hotels['hotel_name'].apply(tag_in_hotel_name)
hotels = hotels.drop(['hotel_name'], axis=1)

# Так как иногда встречалось несколько популярных слов в названии отеля, в результате у некоторых отелей получился список, используем метод explode() что бы распоковать эти списки, и закадируем эти тэги
hotels = hotels.explode('tag_in_hotel_name', ignore_index=False)
hotels = pd.get_dummies(hotels, columns=['tag_in_hotel_name'])


In [ ]:
hotels = hotels.groupby(hotels.index).max()

In [ ]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 515402 entries, 0 to 515737
Data columns (total 33 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   additional_number_of_scoring                515402 non-null  int64  
 1   review_date                                 515402 non-null  object 
 2   average_score                               515402 non-null  float64
 3   reviewer_nationality                        515402 non-null  object 
 4   negative_review                             515402 non-null  object 
 5   review_total_negative_word_counts           515402 non-null  int64  
 6   total_number_of_reviews                     515402 non-null  int64  
 7   positive_review                             515402 non-null  object 
 8   review_total_positive_word_counts           515402 non-null  int64  
 9   total_number_of_reviews_reviewer_has_given  515402 non-null  int64  
 

Аналогично обработаем признаки с негативными и позивными отзывами.

In [ ]:
list_tag_in_negative_review = [] # Список популярных негативных слов

full_tag_in_negative_review = {} # Словарь всех негативных слов

# Напишем функцию приводящую к нижнему регистру(на всякий случай), и создающая список по пробелу, и добавляющая каждое уникальное слово и кол-во его повторений во всех названий отелей
def negative_review_remote(agr):
    name = agr.lower()
    res = name.split(' ')
    for i in res:
        if i not in full_tag_in_negative_review:
            full_tag_in_negative_review[i] = 1
        else:
            full_tag_in_negative_review[i] += 1

hotels['negative_review'].apply(negative_review_remote)

# Так как слов очень много, оставим только те, которые встречаются больше 40000 раз
for i in full_tag_in_negative_review:
    if full_tag_in_negative_review[i] > 40000:
        list_tag_in_negative_review.append(i)

print(len(list_tag_in_negative_review))

36


In [ ]:
list_tag_in_negative_review

['',
 'have',
 'a',
 'in',
 'the',
 'hotel',
 'that',
 'but',
 'not',
 'you',
 'at',
 'to',
 'from',
 'as',
 'there',
 'no',
 'is',
 'was',
 'room',
 'negative',
 'they',
 't',
 'with',
 'of',
 'on',
 'it',
 'be',
 'for',
 'breakfast',
 'and',
 'i',
 'very',
 'small',
 'we',
 'had',
 'were']

Оставляем только наиболее значиемые слова для нас.

In [ ]:
top_tag_in_negative_review = [
    'not',
    'no',
    'room',
    'negative',
    'breakfast',
    'very',
    'small',
    'staff',
    'nothing'
]

In [ ]:
# Напишем функцию которая будет брать поочередно слово из отзыва и проверять, находится ли оно в списке топовых слов, и возвращать только слова которые наиболее популярны.
def tag_in_negative_review(agr):
    name = agr.lower()
    res = name.split(' ')
    result = []
    for i in res:
        if i in top_tag_in_negative_review:
            if i not in result:
                result.append(i)
    if len(result) == 0: # добавил проверку на то есть ли в отзыве популрные слова, если нет, возвращать пустое значение.
        return None
    else:
        return result
    
# применяем нашу функцию и сразу удаляем обработанный признак.
hotels['tag_in_negative_review'] = hotels['negative_review'].apply(tag_in_negative_review)
hotels = hotels.drop(['negative_review'], axis=1)

# Так как иногда встречалось несколько популярных слов в названии отеля, в результате у некоторых отелей получился список, используем метод explode() что бы распоковать эти списки, и закадируем эти тэги
hotels = hotels.explode('tag_in_negative_review', ignore_index=False)
hotels = pd.get_dummies(hotels, columns=['tag_in_negative_review'])

In [ ]:
hotels = hotels.groupby(hotels.index).max()

In [ ]:
list_tag_in_positive_review = [] # Список позитивных популярных слов

full_tag_in_positive_review = {} # Словарь позитивных всех слов

# Напишем функцию приводящую к нижнему регистру(на всякий случай), и создающая список по пробелу, и добавляющая каждое уникальное слово и кол-во его повторений во всех названий отелей
def positive_review_remote(agr):
    name = agr.lower()
    res = name.split(' ')
    for i in res:
        if i not in full_tag_in_positive_review:
            full_tag_in_positive_review[i] = 1
        else:
            full_tag_in_positive_review[i] += 1

hotels['positive_review'].apply(positive_review_remote)

# Так как слов очень много, оставим только те, которые встречаются больше 40000 раз
for i in full_tag_in_positive_review:
    if full_tag_in_positive_review[i] > 40000:
        list_tag_in_positive_review.append(i)

print(len(list_tag_in_positive_review))

33


In [ ]:
list_tag_in_positive_review

['',
 'hotel',
 'was',
 'great',
 'clean',
 'friendly',
 'staff',
 'breakfast',
 'with',
 'good',
 'nice',
 'room',
 'in',
 'the',
 'is',
 'to',
 'a',
 'and',
 'from',
 'no',
 'of',
 'it',
 'excellent',
 'location',
 'bed',
 'comfortable',
 'for',
 'i',
 'rooms',
 'very',
 'we',
 'helpful',
 'were']

In [ ]:
top_tag_in_positive_review = [
    'great',
    'friendly',
    'staff',
    'breakfast',
    'with',
    'good',
    'nice',
    'room',
    'positive',
    'location',
    'comfortable'
]

In [ ]:
# Напишем функцию которая будет брать поочередно слово из отзыва и проверять, находится ли оно в списке топовых слов, и возвращать только слова которые наиболее популярны.
def tag_in_positive_review(agr):
    name = agr.lower()
    res = name.split(' ')
    result = []
    for i in res:
        if i in top_tag_in_positive_review:
            if i not in result:
                result.append(i)
    if len(result) == 0: # добавил проверку на то есть ли в отзыве популрные слова, если нет, возвращать пустое значение.
        return None
    else:
        return result
    
# применяем нашу функцию и сразу удаляем обработанный признак.
hotels['tag_in_positive_review'] = hotels['positive_review'].apply(tag_in_positive_review)
hotels = hotels.drop(['positive_review'], axis=1)

# Так как иногда встречалось несколько популярных слов в названии отеля, в результате у некоторых отелей получился список, используем метод explode() что бы распоковать эти списки, и закадируем эти тэги
hotels = hotels.explode('tag_in_positive_review', ignore_index=False)
hotels = pd.get_dummies(hotels, columns=['tag_in_positive_review'])

In [ ]:
hotels = hotels.groupby(hotels.index).max()

In [ ]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 515402 entries, 0 to 515737
Data columns (total 51 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   additional_number_of_scoring                515402 non-null  int64  
 1   review_date                                 515402 non-null  object 
 2   average_score                               515402 non-null  float64
 3   reviewer_nationality                        515402 non-null  object 
 4   review_total_negative_word_counts           515402 non-null  int64  
 5   total_number_of_reviews                     515402 non-null  int64  
 6   review_total_positive_word_counts           515402 non-null  int64  
 7   total_number_of_reviews_reviewer_has_given  515402 non-null  int64  
 8   tags                                        515402 non-null  object 
 9   days_since_review                           515402 non-null  object 
 

Теперь займемся признаком с датой, возьмем из нее сезон в которых отдыхал ревьювер.

In [ ]:
# Приведем признак к формату to_datetime.
hotels['review_date'] = pd.to_datetime(hotels['review_date'])

In [ ]:
# Пишем функцию выделяющую сезон.
def season (agr):
    if agr == 12 or agr ==1 or agr ==2:
        return 1
    elif agr == 3 or agr ==4 or agr ==5:
        return 2
    elif agr == 6 or agr ==7 or agr ==8:
        return 3
    else:
        return 4

In [ ]:
# Применяем функцию и удаляем старый признак
hotels['season'] = hotels['review_date'].dt.month.apply(season)
hotels = hotels.drop('review_date',axis=1)
# Закадируем признак с сезоном
hotels = pd.get_dummies(hotels, columns=['season'])

Займемся признаком с тэгами

In [ ]:
# немного отформатируем значения признака, что бы можно было обработать методом explode()
def rem_tag(agr):
    agr = agr[2:-2]
    res = agr.strip().split(' \', \'')
    return res


In [ ]:
hotels['tags'] = hotels['tags'].apply(rem_tag)

In [ ]:
hotels = hotels.explode('tags', ignore_index=False)

In [ ]:
# Смотрим на самые популярные тэги
hotels['tags'].value_counts().head(15)

Leisure trip                       416407
 Submitted from a mobile device    307458
 Couple                            246313
 Stayed 1 night                    193557
 Stayed 2 nights                   133846
 Solo traveler                     104183
 Stayed 3 nights                    95754
Business trip                       82689
 Group                              63757
 Family with young children         58761
 Stayed 4 nights                    47774
 Double Room                        35207
 Standard Double Room               32246
 Superior Double Room               31359
 Family with older children         25400
Name: tags, dtype: int64

In [ ]:
# возьмем самые популярные которые могут влиять на оценку.
list_tags = [
    'leisure trip', 
    'submitted from a mobile device', 
    'couple', 
    'stayed 1 night', 
    'stayed 2 nights', 
    'solo traveler', 
    'stayed 3 nights', 
    'business trip', 
    'group', 
    'family with young children'
]

In [ ]:
# напишем функцию которая оставит только самые популярные тэги
def clear_tag(agr):
    res = agr.strip().lower()
    if res in list_tags: # проверка тэга в списке популярности.
        return res
    else:
        return 'other'

In [ ]:
# применяем функцию, и сразу кодируем признак.
hotels['tags'] = hotels['tags'].apply(clear_tag)
hotels = pd.get_dummies(hotels, columns=['tags'])

In [ ]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2355078 entries, 0 to 515737
Data columns (total 64 columns):
 #   Column                                      Dtype  
---  ------                                      -----  
 0   additional_number_of_scoring                int64  
 1   average_score                               float64
 2   reviewer_nationality                        object 
 3   review_total_negative_word_counts           int64  
 4   total_number_of_reviews                     int64  
 5   review_total_positive_word_counts           int64  
 6   total_number_of_reviews_reviewer_has_given  int64  
 7   days_since_review                           object 
 8   sample                                      int64  
 9   reviewer_score                              float64
 10  distance_to_centre_in_km                    float64
 11  City_Amsterdam                              uint8  
 12  City_Barcelona                              uint8  
 13  City_London                 

Так как метод explode конечно хорош, но он создает множество дополнительный строк, поэтому необходимо сгруппировать наш фрейм по индексу.


In [ ]:
hotels = hotels.groupby(hotels.index).max()

Признак days_since_review преобразуем в числовой формат для дальнейшей нормализации.

In [ ]:
def number(agr):
    res = agr.split(' ')
    return int(res[0])

In [ ]:
hotels['days_since_review'] = hotels['days_since_review'].apply(number)

Облегчим наш фрейм что бы занимал меньше места, и быстрее работал в дальнейшем.

In [ ]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 515402 entries, 0 to 515737
Data columns (total 64 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   additional_number_of_scoring                515402 non-null  int64  
 1   average_score                               515402 non-null  float64
 2   reviewer_nationality                        515402 non-null  object 
 3   review_total_negative_word_counts           515402 non-null  int64  
 4   total_number_of_reviews                     515402 non-null  int64  
 5   review_total_positive_word_counts           515402 non-null  int64  
 6   total_number_of_reviews_reviewer_has_given  515402 non-null  int64  
 7   days_since_review                           515402 non-null  int64  
 8   sample                                      515402 non-null  int64  
 9   reviewer_score                              515402 non-null  float64
 

In [ ]:
import numpy as np

In [ ]:
def int_8(i):
    return np.uint8(i)
def float_16(i):
    return np.float16(i)
def int_16(i):
    return np.uint16(i)

In [ ]:
hotels['distance_to_centre_in_km']=hotels['distance_to_centre_in_km'].apply(float_16)
hotels['reviewer_score']=hotels['reviewer_score'].apply(float_16)
hotels['total_number_of_reviews_reviewer_has_given']=hotels['total_number_of_reviews_reviewer_has_given'].apply(int_16)
hotels['review_total_positive_word_counts']=hotels['review_total_positive_word_counts'].apply(int_16)
hotels['total_number_of_reviews']=hotels['total_number_of_reviews'].apply(int_16)
hotels['review_total_negative_word_counts']=hotels['review_total_negative_word_counts'].apply(int_16)
hotels['average_score']=hotels['average_score'].apply(float_16)
hotels['additional_number_of_scoring']=hotels['additional_number_of_scoring'].apply(int_16)
hotels['days_since_review']=hotels['days_since_review'].apply(int_16)
hotels['sample']=hotels['sample'].apply(int_8)

In [ ]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 515402 entries, 0 to 515737
Data columns (total 64 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   additional_number_of_scoring                515402 non-null  int16  
 1   average_score                               515402 non-null  float16
 2   reviewer_nationality                        515402 non-null  object 
 3   review_total_negative_word_counts           515402 non-null  int16  
 4   total_number_of_reviews                     515402 non-null  int16  
 5   review_total_positive_word_counts           515402 non-null  int16  
 6   total_number_of_reviews_reviewer_has_given  515402 non-null  int16  
 7   days_since_review                           515402 non-null  int16  
 8   sample                                      515402 non-null  int8   
 9   reviewer_score                              515402 non-null  float16
 

In [ ]:
hotels.head(10)

,additional_number_of_scoring,average_score,reviewer_nationality,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,days_since_review,sample,reviewer_score,...,tags_couple,tags_family with young children,tags_group,tags_leisure trip,tags_other,tags_solo traveler,tags_stayed 1 night,tags_stayed 2 nights,tags_stayed 3 nights,tags_submitted from a mobile device
0,904,8.101562,United Kingdom,52,16670,62,1,13,0,0.0,...,1,0,0,1,1,0,0,1,0,0
1,612,8.601562,Belgium,10,5018,0,7,234,0,0.0,...,0,0,1,1,1,0,1,0,0,0
2,46,8.296875,Sweden,3,351,5,15,616,0,0.0,...,0,0,0,0,1,1,1,0,0,1
3,241,9.101562,United States of America,0,1543,27,9,656,0,0.0,...,1,0,0,1,1,0,1,0,0,0
4,834,9.101562,United Kingdom,0,4687,7,2,444,0,0.0,...,0,0,0,0,1,1,0,1,0,1
5,709,8.203125,Ecuador,7,2995,26,3,721,0,0.0,...,0,0,0,0,1,1,1,0,0,0
6,1005,8.296875,United Kingdom,35,3469,23,11,350,0,0.0,...,1,0,0,1,1,0,1,0,0,0
7,704,8.296875,Netherlands,52,2443,42,38,723,0,0.0,...,0,0,0,0,1,1,0,0,1,1
8,211,8.898438,Ireland,8,2412,7,1,404,0,0.0,...,0,0,1,0,1,0,1,0,0,0
9,61,7.398438,Canada,18,334,25,1,673,0,0.0,...,1,0,0,0,1,0,0,1,0,1


Удалим признак с тэгом tags_other так как он почти во всех строках отмечается

In [ ]:
hotels['tags_other'].value_counts()

1    515223
0       179
Name: tags_other, dtype: int64

In [ ]:
hotels = hotels.drop('tags_other',axis=1)

Закадируем признак национальности, какими бы мы не были толерантными, национальность тоже может влиять на оценку.

In [ ]:
# Создадим пустой словарь в который добавим все нацональности и кол-во
nationality = {}
def list_reviewer_nationality(agr):
    res = agr.lower()
    res = res.strip()
    if res not in nationality:
        nationality[res] = 1
    else:
        nationality[res] += 1
  
    return res

hotels['reviewer_nationality']=hotels['reviewer_nationality'].apply(list_reviewer_nationality)

# далее создаем список национальностей, которые встречаются более 7000 раз
list_nationality = []

for i in nationality:
  if nationality[i] > 7000:
    list_nationality.append(i)

In [ ]:
# Пишем функцию оставляющую только популярную национальность, и кодируем признак.
def clear_reviewer_nationality(agr):
  if agr not in list_nationality:
    return 'other'
  else:
    return agr

hotels['reviewer_nationality'] = hotels['reviewer_nationality'].apply(clear_reviewer_nationality)
hotels = pd.get_dummies(hotels, columns=['reviewer_nationality'])

In [ ]:
from sklearn import preprocessing

# Для графиков
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
hotels.describe()

,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,days_since_review,sample,reviewer_score,distance_to_centre_in_km,...,reviewer_nationality_france,reviewer_nationality_germany,reviewer_nationality_ireland,reviewer_nationality_netherlands,reviewer_nationality_other,reviewer_nationality_saudi arabia,reviewer_nationality_switzerland,reviewer_nationality_united arab emirates,reviewer_nationality_united kingdom,reviewer_nationality_united states of america
count,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,512149.000000,...,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000
mean,498.288753,NaN,18.539901,2744.281231,17.777527,7.165120,354.404213,0.749892,NaN,NaN,...,0.014115,0.015394,0.028752,0.017000,0.269060,0.017355,0.016826,0.019849,0.475664,0.068655
std,500.624467,0.000000,29.691326,2317.869392,21.803563,11.038862,208.913902,0.433075,0.000000,NaN,...,0.117966,0.123113,0.167110,0.129272,0.443472,0.130592,0.128618,0.139480,0.499408,0.252867
min,1.000000,5.199219,0.000000,43.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,169.000000,8.101562,2.000000,1161.000000,5.000000,1.000000,175.000000,0.000000,0.000000,1.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,342.000000,8.398438,9.000000,2134.000000,11.000000,3.000000,353.000000,1.000000,7.898438,2.599609,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,660.000000,8.796875,23.000000,3633.000000,22.000000,8.000000,527.000000,1.000000,9.601562,4.101562,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,2682.000000,9.796875,408.000000,16670.000000,395.000000,355.000000,730.000000,1.000000,10.000000,1033.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
r_scaler = preprocessing.RobustScaler()
col_names = [
    'additional_number_of_scoring',
    'review_total_negative_word_counts',
    'review_total_positive_word_counts',
    'total_number_of_reviews_reviewer_has_given',
    'days_since_review',
    'distance_to_centre_in_km'
]
hotels_ = r_scaler.fit_transform(hotels[col_names])

hotels_ = pd.DataFrame(hotels_, columns=col_names)

hotels[col_names] = hotels_[col_names]
hotels.describe()

,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,days_since_review,sample,reviewer_score,distance_to_centre_in_km,...,reviewer_nationality_france,reviewer_nationality_germany,reviewer_nationality_ireland,reviewer_nationality_netherlands,reviewer_nationality_other,reviewer_nationality_saudi arabia,reviewer_nationality_switzerland,reviewer_nationality_united arab emirates,reviewer_nationality_united kingdom,reviewer_nationality_united states of america
count,515066.000000,515402.000000,515066.000000,515402.000000,515066.000000,515066.000000,515066.000000,515402.000000,515402.000000,511814.000000,...,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000
mean,0.318357,NaN,0.454270,2744.281231,0.398614,0.595046,0.003983,0.749892,NaN,0.248842,...,0.014115,0.015394,0.028752,0.017000,0.269060,0.017355,0.016826,0.019849,0.475664,0.068655
std,1.019619,0.000000,1.413972,2317.869392,1.282397,1.577140,0.593506,0.433075,0.000000,1.472501,...,0.117966,0.123113,0.167110,0.129272,0.443472,0.130592,0.128618,0.139480,0.499408,0.252867
min,-0.694501,5.199219,-0.428571,43.000000,-0.647059,-0.285714,-1.002841,0.000000,0.000000,-0.999249,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.352342,8.101562,-0.333333,1161.000000,-0.352941,-0.285714,-0.505682,0.000000,0.000000,-0.422673,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,8.398438,0.000000,2134.000000,0.000000,0.000000,0.000000,1.000000,7.898438,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.647658,8.796875,0.666667,3633.000000,0.647059,0.714286,0.494318,1.000000,9.601562,0.577327,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,4.765784,9.796875,19.000000,16670.000000,22.588236,50.285713,1.071023,1.000000,10.000000,396.069824,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
from sklearn.ensemble import RandomForestRegressor 


In [ ]:
import random

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
hotels.info()

In [ ]:
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score']  
      
    # Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  
      
    # Наборы данных с меткой "train" будут использоваться для обучения модели, "test" — для тестирования.  
    # Для тестирования мы будем использовать 25 % от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
      
    # Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
    # Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
    # Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
    # Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)

ValueError: ignored

In [ ]:
hotels.iloc[hotels['average_score'] == None]

NotImplementedError: ignored

In [ ]:
hotels.describe(include = "all")

,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,days_since_review,sample,reviewer_score,distance_to_centre_in_km,...,reviewer_nationality_france,reviewer_nationality_germany,reviewer_nationality_ireland,reviewer_nationality_netherlands,reviewer_nationality_other,reviewer_nationality_saudi arabia,reviewer_nationality_switzerland,reviewer_nationality_united arab emirates,reviewer_nationality_united kingdom,reviewer_nationality_united states of america
count,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,512149.000000,...,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000,515402.000000
mean,498.288753,NaN,18.539901,2744.281231,17.777527,7.165120,354.404213,0.749892,NaN,NaN,...,0.014115,0.015394,0.028752,0.017000,0.269060,0.017355,0.016826,0.019849,0.475664,0.068655
std,500.624467,0.000000,29.691326,2317.869392,21.803563,11.038862,208.913902,0.433075,0.000000,NaN,...,0.117966,0.123113,0.167110,0.129272,0.443472,0.130592,0.128618,0.139480,0.499408,0.252867
min,1.000000,5.199219,0.000000,43.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,169.000000,8.101562,2.000000,1161.000000,5.000000,1.000000,175.000000,0.000000,0.000000,1.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,342.000000,8.398438,9.000000,2134.000000,11.000000,3.000000,353.000000,1.000000,7.898438,2.599609,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,660.000000,8.796875,23.000000,3633.000000,22.000000,8.000000,527.000000,1.000000,9.601562,4.101562,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,2682.000000,9.796875,408.000000,16670.000000,395.000000,355.000000,730.000000,1.000000,10.000000,1033.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
